# Week 2 - Working with Excel files


Overview:
* Reading and writing Excel files with Pandas
* Reading and writing Excel files with openpyxl
* Aside: Accessing the SQLite3 database with Python
* Aside 2: The *Dataset* library
* Use Case 1: Translate spreadsheet into database
* Use Case 2: Update spreadsheet from database

## Reading and writing Excel files with Pandas

In what could be considered by some to be a cruel trick, we will only mention here that Pandas has a read_excel() function that will give you a dataframe made from the Excel data. As with all things, there are pros and cons to accessing Excel data this way but it is out of the scope of this FTE to spend time enumerating them. 

## Reading and writing Excel files with openpyxl

Openpyxl is a Python library devoted to reading and manipulating Excel files. I believe Anaconda Python has it installed but if you don't have it, you can install it with conda or pip. 

Just as Excel's main unit of work is the workbook, openpyxl has a `load_workbook()` function. 

In [1]:
from openpyxl import load_workbook

In [2]:
workbook = load_workbook(filename="data/aapl_HistoricalQuotes_10yr.xlsx")

We can look at the workbook's sheet names:

In [3]:
workbook.sheetnames

['aapl_HistoricalQuotes_10yr']

And decide to use the active one. 

It is worth noting that it is, of course, possible to use other sheets but we only have one in this file. 

In [4]:
sheet = workbook.active

sheet.title

'aapl_HistoricalQuotes_10yr'

We can reference individual cells in the sheet by using dictionary notation, with the row and column put together in the square brackets. 

That sounded much more confusing than it is. To look at row 1's A column, we can do this:

In [5]:
sheet['A1']

<Cell 'aapl_HistoricalQuotes_10yr'.A1>

Well, that wasn't very useful. Let's try looking at the value.

In [6]:
sheet['A1'].value

'date'

That's better!

We can also ask for cells by row and column:

In [7]:
sheet.cell(row=6, column=5).value

205.72

We can also work with ranges:

In [8]:
sheet["A1:C2"]

((<Cell 'aapl_HistoricalQuotes_10yr'.A1>,
  <Cell 'aapl_HistoricalQuotes_10yr'.B1>,
  <Cell 'aapl_HistoricalQuotes_10yr'.C1>),
 (<Cell 'aapl_HistoricalQuotes_10yr'.A2>,
  <Cell 'aapl_HistoricalQuotes_10yr'.B2>,
  <Cell 'aapl_HistoricalQuotes_10yr'.C2>))

### Iterating

If you took MSDE 620, you may remember we discussed "iterators," and "iterating," and other forms of the word. The tl;dr version is that iterating means using a loop. 

As we saw up above, it is possible to grab cell objects but for our purposes, getting the values is more important. Here is an example:

In [9]:
for value in sheet.iter_rows(min_row=1,
                             max_row=5,
                             values_only=True):
    print(value)

('date', 'close', 'volume', 'open', 'high', 'low')
(datetime.datetime(2019, 9, 4, 0, 0), 209.19, 19216820, 208.39, 209.48, 207.32)
(datetime.datetime(2019, 9, 3, 0, 0), 205.7, 20059570, 206.43, 206.98, 204.22)
(datetime.datetime(2019, 8, 30, 0, 0), 208.74, 21162560, 210.16, 210.45, 207.2)
(datetime.datetime(2019, 8, 29, 0, 0), 209.01, 21007650, 208.5, 209.32, 206.655)


In [10]:
for value in sheet.iter_rows(min_row=1,
                             max_row=5,
                             values_only=True):
    print(f'{value[0]}  {value[1]}   {value[2]}   {value[3]}   {value[4]}   {value[5]}   ')

date  close   volume   open   high   low   
2019-09-04 00:00:00  209.19   19216820   208.39   209.48   207.32   
2019-09-03 00:00:00  205.7   20059570   206.43   206.98   204.22   
2019-08-30 00:00:00  208.74   21162560   210.16   210.45   207.2   
2019-08-29 00:00:00  209.01   21007650   208.5   209.32   206.655   


Now that we've learned how to get to the data in the spreadsheet, let's learn the basics of SQL and relational databases. 
<hr>

<img align="right" style="padding-right:10px;" src="figures/sqlite370_banner.gif" width=200><br>
## Aside: Accessing the SQLite3 database with Python
<div style="text-align: right">
https://www.sqlite.org/index.html
</div>
<br>
SQLite is a file-based database, meaning there is no "Database Management" program, like Oracle or MySQL. There is only the data file on disk that needs to be loaded. This is the beauty of SQLite -- you can include it as a storage solution in a program without having to install an RDBMS. 

When accessing a database programmatically, regardless of language, the same basic set of steps need to be performed, at least to start:

1. Connect to database - this will give a "connection object" or **handle** to access the DB internals.
2. Use the connection object's functions to run queries -- **save results to a variable**.
3. The result variable will be organized as rows and columns and can be iterated through.

First, we will create two DataFrames from dictionaries that represent our employee and department tables. Then we will create a new `emp.db` database file and tell the DataFrames to convert (write) them to SQLite tables. 

**employees**

emp_id | emp_name | dept_id
--|-----|-----------
1 | Tom | 1
2 | Mary | 2
3 | John | 3
4 | Tim | 1
5 | Jenny | 

NOTE: Jenny is a new hire not assigned a department yet.<br>
**departments**

dept_id | dept_name
--------|----------
1 | HR
2 | Development
3 | Marketing


In [11]:
import sqlite3

In [12]:
import pandas as pd
import numpy as np

# Have to put a 0 in for Jenny, then adjust it.
emp_df = pd.DataFrame({'emp_id':[1,2,3,4,5], 'emp_name':['Tom', 'Mary','John', 'Tim', 'Jenny'], 'dept_id':[1,2, 3, 1, 0] })
dept_df = pd.DataFrame({'dept_id':[1,2,3], 'dept_name':['HR','Development', 'Marketing']})

# Workaround to get a null into Jenny's dept_id
# emp_df.loc[emp_df['emp_name'] == 'Jenny', 'dept_id'] = np.nan

print(f'{emp_df}\n\n{dept_df}')

   emp_id emp_name  dept_id
0       1      Tom        1
1       2     Mary        2
2       3     John        3
3       4      Tim        1
4       5    Jenny        0

   dept_id    dept_name
0        1           HR
1        2  Development
2        3    Marketing


**Quick aside:** If we try to create tables in a database where they already exist, we will get a ValueError, like we see below (if you are running this code and the cell below doesn't give an error, run it twice).

This can be handled in two ways:

* Use __exception handling__ e.g. a try/except block (below)
* Drop the table(s) and recreate (example in Use Cases).

In [13]:
# Making connection to database
# con will be our connection "handle" 
con = sqlite3.connect("data/emp.db")

emp_df.to_sql('employees', con)
dept_df.to_sql('departments', con)


ValueError: Table 'employees' already exists.

A try/except block works exactly as it sounds -- **try** to run some risky code. Catch and handle any **except**ion that happens. 

Virtually every modern language has some form of exception handling to catch errors. 

**NOTE:** Exceptions are different from code syntax errors. An exception happens when the code is technically correct but something unexpected happens -- wrong type of value in a variable, file exists or doesn't exist (depending on operation), etc. 

Here is how we could handle the database error above:

```
try:
    emp_df.to_sql('employees', con)
    dept_df.to_sql('departments', con)
except ValueError as v_error:
    print(v_error)
```

Try to create the tables, but if we get a ValueError, print the error.

In [14]:
try:
    emp_df.to_sql('employees', con)
    dept_df.to_sql('departments', con)
except ValueError as v_error:
    print(v_error)

Table 'employees' already exists.


## Accessing DB Data from Python

### DDL and DML

SQL is actually composed of two languages:
* DDL - Data Definition Language, used to create tables, etc.
* DML - Data Manipulation Language, used to insert, update and retrieve data from the tables.

We were able to skip the table creation phase above using Pandas to write our dataframes. 

When working with databases, by far the most common operation is retrieving (SELECTing) data, followed by INSERTing and UPDATEing. Let's look at SELECT first.

### SQL SELECT

When SQL first came out, lower case letters hadn't been invented yet (joke!), so by convention, SQL keywords like SELECT, FROM, WHERE, etc. are written in all caps. However, to an old Unix/C programmer like me, caps make my skin crawl. In reality, the database won't care about caps or no caps so I may forget to use them.

SELECT statements are composed of 3 parts:

1. The keyword SELECT followed by a list of columns to display
2. The keyword FROM followed by the table name(s) from which to get the data
3. The keyword WHERE followed by logical statement that acts as a **filter** for the data. We'll look at this in depth in a moment.

Written in a more "normal" form, it looks like this:

```
SELECT column a, column b, column c 
FROM table
WHERE column = value;
```

To use our example "employees" table from above: 
`SELECT emp_id, emp_name FROM employees where dept_id = 1;`

This would return to us a table of results showing 

|emp_id|emp_name|
|------|--------|
| 1 | Tom |
| 4 | Tim |

That gives us enough information to ask the database to tell us all its table names.

### Querying the master table 

If this was the first time seeing the database, you might not know much about it and need to get some info.

All DB systems have some sort of "Master" table(s) that keeps track of the database **schema** -- structural info about the DB architecture. 

A really useful query is to ask the "Master" for all the (other) table names. We will do this using the connection object's `execute()` function to pass raw SQL to the database. 

In [15]:
cursor = con.execute('select name from sqlite_master where type = "table";')

Above, I'm asking the `sqlite_master` table to return the `name` column contents for every row whose `type` column has the word "table" in it. 

Go back and read that sentence again and make sure you understand how my verbal description matches the SQL. It is basically a long-winded way of saying 'return all table names from the sqlite_master table.'

Results are returned to your Python code as `cursor` objects. 

In [16]:
type(cursor)

sqlite3.Cursor

The line above shows that the `cursor` variable containing the query result is a **Cursor** data type. Database cursors are the table-like returns from a query. To see the query results, we can just ask the cursor to `fetchall()` results.

In [17]:
cursor.fetchall()

[('employees',), ('departments',)]

This tells us that we have 2 tables in our database, **"employees"** and **"departments"**. 

You could also get a list of column names from the master table but then you wouldn't necessarily know what table they live in. Instead, let's query a table and use the result cursor's `description` property. The first entry in the description is the column name.

In [18]:
cursor = con.execute('select * from employees;')

In [19]:
cursor.description

(('index', None, None, None, None, None, None),
 ('emp_id', None, None, None, None, None, None),
 ('emp_name', None, None, None, None, None, None),
 ('dept_id', None, None, None, None, None, None))

So, if we just want a list of column names for the employee table, we could use one of three different approaches: 
* a for loop
* a list comprehension 
* map with lambda function 

We will look at all three.

In [20]:
# Regular for loop
for col in cursor.description:
    print(col[0])

index
emp_id
emp_name
dept_id


In [21]:
# List comprehension 
emp_cols = [col[0] for col in cursor.description]

In [22]:
emp_cols

['index', 'emp_id', 'emp_name', 'dept_id']

In [23]:
# map function - Python 3 map is a special type
# so we wrap it in a list to be able to "look inside" it.
emp_cols2 = list(map(lambda x: x[0], cursor.description))

emp_cols2

['index', 'emp_id', 'emp_name', 'dept_id']

Now that we know how to list tables and columns, we can select data from individual tables or do a join to replace some of those xxx_id columns with actual words.

First, let's do some basic `select` statements.

In SQL, the '\*' (asterisk - is a wildcard character -- it means 'Give me everything' and the most basic query you can do is:

`SELECT * FROM table;`

Notice there is no 'WHERE' clause. That means the query will return **all** rows in the table. 

Let's see it in action:

In [24]:
data = con.execute('select * from employees;')
for line in data:
    print(line)

(0, 1, 'Tom', 1)
(1, 2, 'Mary', 2)
(2, 3, 'John', 3)
(3, 4, 'Tim', 1)
(4, 5, 'Jenny', 0)


Notice how that gave us all rows and all columns.

In [25]:
data = con.execute('select employees.emp_name, d.dept_name from employees join departments d on employees.dept_id = d.dept_id;')

In [26]:
for line in data:
    print(f'{line[0]}\t\t{line[1]}')

Tom		HR
Mary		Development
John		Marketing
Tim		HR


You can use tricks like above to get the column names.

Let's do a search for all employees whose name starts with 'T'.

In [27]:
con.execute('select emp_name from employees where emp_name like "T%";').fetchall()

[('Tom',), ('Tim',)]

### Inserting data

You can put data into a database with the `INSERT` statement. The syntax is a bit different from a `SELECT`. 

```
INSERT into table_name (column1, column2, column3, etc.) VALUES (value1, value2, value3, etc.);
```
The column list does not have to be the same order as the actual table, but, the values must be in the same order as the columns in the statement. Let's INSERT a new employee:

| Index | emp_id | emp_name | dept_id |
|-------|--------|----------|---------|
| 5     | 6      | Jeff Davis | 1     |

**NOTE:** Pandas inserted the index number automatically but normally we wouldn't use it. Similarly, emp_id would be auto-generated by the database at time of insertion, not assigned by the user. 

The INSERT statement for that employee would look like this:

```
INSERT into employees ('index', 'emp_id', 'emp_name', 'dept_id') VALUES (5, 6, 'Jeff Davis', 1);
```
Now, let's actually do that in Python. 

In [28]:
con.execute("INSERT into employees ('index', 'emp_id', 'emp_name', 'dept_id') VALUES (5, 6, 'Jeff Davis', 1);").fetchall()

[]

Notice that a CURSOR is returned but it is empty. Let's look at the table and verify our employee was inserted. 

In [29]:
con.execute('select * from employees;').fetchall()

[(0, 1, 'Tom', 1),
 (1, 2, 'Mary', 2),
 (2, 3, 'John', 3),
 (3, 4, 'Tim', 1),
 (4, 5, 'Jenny', 0),
 (5, 6, 'Jeff Davis', 1)]

Usually, if you try to insert the same data more than once, a well-designed database will give an error. In that case, you should `DELETE` the row before trying to `INSERT` again. Syntax for the `DELETE` statement is as follows:

```
DELETE from table_name WHERE condition;
```
**IMPORTANT NOTE:** Notice the `DELETE` command has a `WHERE` clause. This is to filter/select only the proper rows for deletion. **If you forget the `WHERE` clause, it will delete all the rows in the table!** Trust me. I've done it. Not fun to explain to your boss!

In [30]:
con.execute('delete from employees where emp_id = 6;')
con.execute('select * from employees;').fetchall()

[(0, 1, 'Tom', 1),
 (1, 2, 'Mary', 2),
 (2, 3, 'John', 3),
 (3, 4, 'Tim', 1),
 (4, 5, 'Jenny', 0)]

Let's put Jeff Davis back in, then use an `UPDATE` to change his name. 

In [31]:
con.execute("INSERT into employees ('index', 'emp_id', 'emp_name', 'dept_id') VALUES (5, 6, 'Jeff Davis', 1);")
con.execute('select * from employees;').fetchall()

[(0, 1, 'Tom', 1),
 (1, 2, 'Mary', 2),
 (2, 3, 'John', 3),
 (3, 4, 'Tim', 1),
 (4, 5, 'Jenny', 0),
 (5, 6, 'Jeff Davis', 1)]

Syntax for the `UPDATE` is a bit of a mix of `SELECT` and `INSERT`.

```
UPDATE table_name set column1 = value, column2 = value, etc., WHERE condition;
```

Since you have a `WHERE` clause, you can pick and choose many rows to update, for example if we were updating a state_abbreviation table:

```
UPDATE state_abbreviation set abbrev = 'CO' where abbrev = 'Colo';
```
Notice it is perfectly OK to update the same column that is in the `WHERE` clause. 

If you only want to update one row, you need something unique about that row. Usually, that is where the table's \_id column comes into play. Let's use emp_id to change Jeff Davis to a different department. 

In [32]:
con.execute("UPDATE employees set dept_id = 2 where emp_id = 6;")
con.execute('select * from employees;').fetchall()

[(0, 1, 'Tom', 1),
 (1, 2, 'Mary', 2),
 (2, 3, 'John', 3),
 (3, 4, 'Tim', 1),
 (4, 5, 'Jenny', 0),
 (5, 6, 'Jeff Davis', 2)]

Finally, it is good form to close database connections when done. If you forget, you might be wasting system resources (memory leak, for example).

In [33]:
con.close()

Just knowing that little bit will account for 90% of everything you'll generally have to do with databases.

In a moment, we will look at a couple of use cases where we can put it to work.

<hr>

## Aside 2: The Dataset Library

The optional book, *'Data Wrangling with Python'* mentions the Dataset library (https://dataset.readthedocs.io/en/latest/)to mitigate much of the SQL complexity we see above. Dataset says it is "databases for lazy people."

Let's take a quick look at Dataset so we can use it in the Use Cases.

Dataset can be installed as normal, using conda or pip.

We will import the library and then work with the employees database from above.

In [34]:
import dataset

db = dataset.connect("sqlite:///data/emp.db")

Let's get a listing of tables:

In [35]:
db.tables

['departments', 'employees']

And a listing of columns in the employees table:

In [36]:
db['employees'].columns

['index', 'emp_id', 'emp_name', 'dept_id']

Let's get all the department names

In [37]:
for dept in db['departments']:
    print(dept['dept_name'])

HR
Development
Marketing


Now, what is the department ID for Development?

In [38]:
db['departments'].find_one(dept_name='Development')

OrderedDict([('index', 1), ('dept_id', 2), ('dept_name', 'Development')])

We can insert a new row into a table by passing a dictionary into the `insert()` method. Let's add an employee. First, I want to look at what is in that table. This is feasible since our table is so small.

In [39]:
for emp in db['employees'].all():
    print(emp)

OrderedDict([('index', 0), ('emp_id', 1), ('emp_name', 'Tom'), ('dept_id', 1)])
OrderedDict([('index', 1), ('emp_id', 2), ('emp_name', 'Mary'), ('dept_id', 2)])
OrderedDict([('index', 2), ('emp_id', 3), ('emp_name', 'John'), ('dept_id', 3)])
OrderedDict([('index', 3), ('emp_id', 4), ('emp_name', 'Tim'), ('dept_id', 1)])
OrderedDict([('index', 4), ('emp_id', 5), ('emp_name', 'Jenny'), ('dept_id', 0)])


Now, let's insert Jeff again:

In [40]:
db['employees'].insert(dict(index=5, emp_id=6, emp_name='Jeff', dept_id=2))

True

Now, to double-check it went right:

In [41]:
for emp in db['employees'].all():
    print(emp)

OrderedDict([('index', 0), ('emp_id', 1), ('emp_name', 'Tom'), ('dept_id', 1)])
OrderedDict([('index', 1), ('emp_id', 2), ('emp_name', 'Mary'), ('dept_id', 2)])
OrderedDict([('index', 2), ('emp_id', 3), ('emp_name', 'John'), ('dept_id', 3)])
OrderedDict([('index', 3), ('emp_id', 4), ('emp_name', 'Tim'), ('dept_id', 1)])
OrderedDict([('index', 4), ('emp_id', 5), ('emp_name', 'Jenny'), ('dept_id', 0)])
OrderedDict([('index', 5), ('emp_id', 6), ('emp_name', 'Jeff'), ('dept_id', 2)])


That should be the majority of what we need for the Use Cases.
<hr>

# Use Cases

Note: The first time we run database code from Use Case 1, the SQLite file will be created.

## Use Case 1: Translate spreadsheet into database

Let's assume for a moment that your manager receives a spreadsheet of data every week that you want to save for further analysis. A little Python code that reads directly out of the Excel file and inserts in the database is a great way to go. 

We will use openpyxl and dataset, combining all that we did above.

We have several Excel files in the data directory, each one representing one year's worth of Apple stock data. We will **create** the database here with the 2009 data. It will be your assignment to get all the rest of the data into the database.

Even though there are only 10 files, we want to get in the habit of programmatically building the file names. Think of it this way: today we *only* have 10 files. What if your data set had 100 files? Or 1000? I'll give you some hints as we work through it. 

You may notice that all the file names follow the same pattern: YYYY_aapl_data.xlsx where YYYY is a year value. This was done intentionally and you will often find data files will have some sort of logical naming scheme. Think of it like this: you want to be able to programmatically create the names to read them, and some developer wanted to programmatically create the names to write them. Many times data files will be the result of some automated process that has very little human intervention. 

Looking again at those names, you may notice that the only thing that changes is the year. That means we can create a variable to hold the constant part. Also, we might as well put the database file name into a variable, too:

In [42]:
input_name = '_aapl_data.xlsx'
db_file = 'stock_prices.db'

Now, let's use Dataset to create the connection. Dataset will create the .db file if it doesn't exist but we won't have the normal database safeguards in place to guard against duplicates, so we'll have to be careful about re-running the next few cells multiple times.

Notice how we join a string to the file name variable to get the complete connection info for Dataset. 

**HINT:** This technique might be useful in your homework!!

In [43]:
db = dataset.connect("sqlite:///" + db_file)

At this point, our database should be completely empty. We should confirm that fact and if it **is** empty, we can create the table. If it isn't we should drop and recreate the table. This isn't something we would normally do in a production environment but while we are learning we could be rerunning the code many times and databases generally hate inserting duplicate data.

To accomplish this, I'm going to do just about the simplest check possible. I'm going to ask for the table names and if the list comes back empty, then the database must be empty. If the list has something in it, I'll drop the table.

In [44]:
# Use a for loop just in case someone snuck in a new table on us
if (len(db.tables) > 0):
    for table in db.tables:
        db[table].drop()


In [45]:
# Create an empty table. We'll create the columns after that from the spreadsheet's first row.
# The create_table function returns the table so we automatically have a handle
table = db.create_table("aapl")

OK, now let's open our 2009 data, get the header and create table columns!

In [46]:
year = '2009'
data_file = "data/" + year + input_name
workbook = load_workbook(filename=data_file)
sheet = workbook.active

In [47]:
header = sheet[1]
header[0].value     # This is just to sanity-check that we got the header

'date'

OK, everything is just about ready. We have to give the columns a type when we create them but that isn't a problem since we **inspected the data in a text editor.** All of the columns are floating point except for 'date' which is, predictably, a datetime type. 

For this handful of columns we could have individual create statements, but that's the amateur way to do it! Probably the best way to figure out many column types would be to read the second row and loop through the cells, detecting the type and storing it in a list. The problem is that everything read in from a file will be a string, so we need to detect what type is hiding in the string. Fortunately, the string type has some allies for us. 

`isdigit()` will return `True` if the string is an integer, but will give `False` if there is a decimal point. The `float()` function will throw a ValueError if there are letters in the string, and our dates always have '/' in them. 

That should be enough to figure out what we have. Maybe we should test it.

In [48]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

# Note: These have to be tested in the right order. isfloat() reports True for integers.
def get_type(value):
    if value.isdigit():
        return dataset.types.Integer
    elif isfloat(value):
        return dataset.types.Float
    elif '/' in value:
        return dataset.types.Date
    else:
        return dataset.types.Unicode

Now, let's test the functions:

In [49]:
print(get_type('abc'))
print(get_type('ab.c'))
print(get_type('1.1'))
print(get_type('1'))
print(get_type('1/1/2019'))


<class 'sqlalchemy.sql.sqltypes.Unicode'>
<class 'sqlalchemy.sql.sqltypes.Unicode'>
<class 'sqlalchemy.sql.sqltypes.Float'>
<class 'sqlalchemy.sql.sqltypes.Integer'>
<class 'sqlalchemy.sql.sqltypes.Date'>


OK, that looks pretty good. Let's try it out on a real row.

In [50]:
row2 = sheet[2]
for cell in row2:
    print(f'{cell.value} is {get_type(cell.value)}')

2009/12/31 is <class 'sqlalchemy.sql.sqltypes.Date'>
30.1046 is <class 'sqlalchemy.sql.sqltypes.Float'>
87907426.0000 is <class 'sqlalchemy.sql.sqltypes.Float'>
30.4471 is <class 'sqlalchemy.sql.sqltypes.Float'>
30.4786 is <class 'sqlalchemy.sql.sqltypes.Float'>
30.0800 is <class 'sqlalchemy.sql.sqltypes.Float'>


Seems to work! You might want to keep `get_type()` and `isfloat()` where you can find them. They could come in handy someday.

We are finally ready to create the columns of the database! The procedure is similar to the cell above, except well will use Dataset's `create_column()` function instead of `print()` in the loop. The syntax is:

`table.create_column(name, type)`

Where **name** is the name of the column in the database, and **type** corresponds to a Dataset definition (that also corresponds to types defined in the *very* popular library, SQLAlchemy). This is why we had `get_type()` return the Dataset type.

In [51]:
# Remember: header contains the first row cells that are column names
# the enumerate function outputs a number to the variable index
# index 
for index, col_name in enumerate(header):
    table.create_column(col_name.value, get_type(row2[index].value))

And then check your work:

In [52]:
table.columns

['id', 'date', 'close', 'volume', 'open', 'high', 'low']

ID is a unique number generated by the database. We can ignore it when doing inserts.

Speaking of inserts, it is just about that time. Dataset inserts a dictionary as a row so our task will be to loop through each row of the worksheet and construct a dictionary from the cells. The dictionary keys are the column names (which is conveniently stored in the `table.columns` **list**) and the values are the cell values.

Each worksheet has a `values` property we can use for our loop's iterable to only return values. This property is just a big block of rows so if we want to slice off the header we have to convert it to something list-like:

`tuple(sheet.values)[1:]`

We can build our dictionary from that. Remember we don't have to insert the `id` column so we will slice it and remove the `id`.

In [53]:
from datetime import datetime,date
keys = table.columns[1:]

for values in list(sheet.values)[1:]:
    row = []
    row.append(datetime.strptime(values[0],'%Y/%m/%d').date())
    row = row + list(values[1:])
    d_row = dict(zip(keys,row))
    table.insert(d_row)



As a final check, we can see if the number of worksheet rows matches the number of table rows.

In [54]:
print(f'Rows in worksheet (plus header): {len(sheet["A"])}' )
print(f'Rows in database: {len(table)}')

Rows in worksheet (plus header): 83
Rows in database: 82


Looks good! You'll get a chance to practice with the 2010 to 2019 data files in the lab.

## Use Case 2: Update spreadsheet from database

Now, let's assume the opposite case: you have stock data streaming into a database and your boss wants a spreadsheet every week with year-to-date data.

First, a quick, little function to print all the rows of a worksheet, called `print_rows()`, then we will create a new worksheet and query our database.

In [55]:
def print_rows():
    for row in sheet.iter_rows(values_only=True):
        print(row)

First, let's look at creating a new worksheet and adding a header and data.

In [56]:
from openpyxl import Workbook
from openpyxl.chart import BarChart, Reference

workbook = Workbook() # Create a new workbook
sheet = workbook.active # Get the active worksheet

# Let's create some sample stock data using nested lists
# rows is a list, and each element of rows is a list of cells.
rows = [
    ['date', 'close', 'volume', 'open', 'high', 'low'],
    ['2019/09/04',209.1900,19216820.0000,208.3900,209.4800,207.3200],
    ['2019/09/03',205.7000,20059570.0000,206.4300,206.9800,204.2200],
    ['2019/08/30',208.7400,21162560.0000,210.1600,210.4500,207.2000],
    ['2019/08/29',209.0100,21007650.0000,208.5000,209.3200,206.6550],
    ['2019/08/28',205.5300,15957630.0000,204.1000,205.7200,203.3200],
    ['2019/08/27',204.1600,25897340.0000,207.8600,208.5500,203.5300],
    ['2019/08/26',206.4900,26066130.0000,205.8600,207.1900,205.0573],
]

for row in rows:
    sheet.append(row)

In [57]:
print_rows()

('date', 'close', 'volume', 'open', 'high', 'low')
('2019/09/04', 209.19, 19216820.0, 208.39, 209.48, 207.32)
('2019/09/03', 205.7, 20059570.0, 206.43, 206.98, 204.22)
('2019/08/30', 208.74, 21162560.0, 210.16, 210.45, 207.2)
('2019/08/29', 209.01, 21007650.0, 208.5, 209.32, 206.655)
('2019/08/28', 205.53, 15957630.0, 204.1, 205.72, 203.32)
('2019/08/27', 204.16, 25897340.0, 207.86, 208.55, 203.53)
('2019/08/26', 206.49, 26066130.0, 205.86, 207.19, 205.0573)


It should be fairly obvious at this point how to proceed:

1. Open the database connection (with or without Dataset)
2. Query the database to get a collection of database
3. Convert the db data into a list of lists
4. Create a new workbook and worksheet
5. Get the table column names as a header
6. Append the header and rows to the worksheet
7. Save the worksheet as an Excel file.

Let's go.



In [58]:
import dataset
db = dataset.connect("sqlite:///stock_prices.db")
print(db.tables)

['aapl']


OK, we have our datafile open and found the name of the table. Let's do a couple of queries to see how data comes back to us.

In [59]:
# Just show everything
for i, row in enumerate(db['aapl']):
    if i < 10:
        print(row)

OrderedDict([('id', 1), ('date', datetime.date(2009, 12, 31)), ('close', 30.1046), ('volume', 87907426.0), ('open', 30.4471), ('high', 30.4786), ('low', 30.08)])
OrderedDict([('id', 2), ('date', datetime.date(2009, 12, 30)), ('close', 30.2343), ('volume', 102705781.0), ('open', 29.8328), ('high', 30.2857), ('low', 29.7586)])
OrderedDict([('id', 3), ('date', datetime.date(2009, 12, 29)), ('close', 29.8714), ('volume', 110755363.0), ('open', 30.3757), ('high', 30.3886), ('low', 29.8186)])
OrderedDict([('id', 4), ('date', datetime.date(2009, 12, 28)), ('close', 30.23), ('volume', 160784168.0), ('open', 30.2457), ('high', 30.5643), ('low', 29.9444)])
OrderedDict([('id', 5), ('date', datetime.date(2009, 12, 24)), ('close', 29.8628), ('volume', 125222058.0), ('open', 29.0786), ('high', 29.9071), ('low', 29.05)])
OrderedDict([('id', 6), ('date', datetime.date(2009, 12, 23)), ('close', 28.8714), ('volume', 86118086.0), ('open', 28.75), ('high', 28.9114), ('low', 28.6871)])
OrderedDict([('id', 

OK, it comes back as a type of dictionary. That means we can use dictionary syntax to get the data out of the parentheses.

In [60]:
# Let's format it a bit
for i, row in enumerate(db['aapl']):
    if i < 10:
        print(f"{row['date']} {row['close']} {row['volume']} {row['open']} {row['high']} {row['low']}")

2009-12-31 30.1046 87907426.0 30.4471 30.4786 30.08
2009-12-30 30.2343 102705781.0 29.8328 30.2857 29.7586
2009-12-29 29.8714 110755363.0 30.3757 30.3886 29.8186
2009-12-28 30.23 160784168.0 30.2457 30.5643 29.9444
2009-12-24 29.8628 125222058.0 29.0786 29.9071 29.05
2009-12-23 28.8714 86118086.0 28.75 28.9114 28.6871
2009-12-22 28.6228 87148416.0 28.4914 28.6928 28.38
2009-12-21 28.3186 152166116.0 28.0071 28.5357 27.9528
2009-12-18 27.9186 151863506.0 27.5957 27.929 27.5143
2009-12-17 27.4086 96720359.0 27.7514 27.8571 27.2857


Now, let's run a query to find all the days the stock closed above 30.

In [61]:
high_close = db['aapl'].find(close = {'>=': 27})
for i, row in enumerate(high_close):
    if i < 10:
        print(f"{row['date']} {row['close']} {row['volume']} {row['open']} {row['high']} {row['low']}")

2009-12-31 30.1046 87907426.0 30.4471 30.4786 30.08
2009-12-30 30.2343 102705781.0 29.8328 30.2857 29.7586
2009-12-29 29.8714 110755363.0 30.3757 30.3886 29.8186
2009-12-28 30.23 160784168.0 30.2457 30.5643 29.9444
2009-12-24 29.8628 125222058.0 29.0786 29.9071 29.05
2009-12-23 28.8714 86118086.0 28.75 28.9114 28.6871
2009-12-22 28.6228 87148416.0 28.4914 28.6928 28.38
2009-12-21 28.3186 152166116.0 28.0071 28.5357 27.9528
2009-12-18 27.9186 151863506.0 27.5957 27.929 27.5143
2009-12-17 27.4086 96720359.0 27.7514 27.8571 27.2857


OK, now let's see how to automate that better. OrderedDict has an `items()` function that will give you the key, value pairs out of the dictionary. You can then use a for loop like this:

`for key, value in dict.items()`

and if you just want a list of the values:

`values = [value for key, value in dict.items()]`

Let's apply that to our query to check it.

In [62]:
# Have to run the query each time we use it
high_close = db['aapl'].find(close = {'>=': 30})
for row in high_close:
    vals = [v for k, v in row.items()]
    print(vals)

[1, datetime.date(2009, 12, 31), 30.1046, 87907426.0, 30.4471, 30.4786, 30.08]
[2, datetime.date(2009, 12, 30), 30.2343, 102705781.0, 29.8328, 30.2857, 29.7586]
[4, datetime.date(2009, 12, 28), 30.23, 160784168.0, 30.2457, 30.5643, 29.9444]


That's all we need! We wil use slicing to exclude that index column.

In [63]:
high_close = db['aapl'].find(close = {'>=': 27})
excel_rows = []

# Get the headers
# excel_rows.append(db['aapl'].columns[1:])

for row in high_close:
    vals = [v for k, v in row.items()]
    excel_rows.append(vals[1:])

excel_rows[:5]

[[datetime.date(2009, 12, 31), 30.1046, 87907426.0, 30.4471, 30.4786, 30.08],
 [datetime.date(2009, 12, 30),
  30.2343,
  102705781.0,
  29.8328,
  30.2857,
  29.7586],
 [datetime.date(2009, 12, 29),
  29.8714,
  110755363.0,
  30.3757,
  30.3886,
  29.8186],
 [datetime.date(2009, 12, 28), 30.23, 160784168.0, 30.2457, 30.5643, 29.9444],
 [datetime.date(2009, 12, 24), 29.8628, 125222058.0, 29.0786, 29.9071, 29.05]]

The rest is easy!

In [64]:
workbook = Workbook() # Create a new workbook
sheet = workbook.active # Get the active worksheet

header = db['aapl'].columns[1:]
# The header row comes from the db as a weird type
# so we convert to string
header = [str(v) for v in header]
sheet.append(header)
for row in excel_rows:
    sheet.append(row)
 

The final step is to write the Excel file.

In [65]:
fname = "621_demo.xlsx"

workbook.save(filename=fname)